# Data preparation

In [4]:
import tensorflow as tf
import pandas as pd
import numpy as np

c:\users\mihne\desktop\nn-firewall\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\mihne\desktop\nn-firewall\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\mihne\desktop\nn-firewall\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\mihne\desktop\nn-firewall\venv\lib\site-packages\ten

## Importing the network packets exported from Wireshark

First, we need to import all the sets of before-after packets.

In [117]:
before_firewall_all_1 = pd.read_csv('Data/Set 1/before_firewall_1_flags.csv')
after_firewall_all_1 = pd.read_csv('Data/Set 1/after_firewall_1_flags.csv')

In [118]:
before_firewall_all_2 = pd.read_csv('Data/Set 2/before_firewall_2_flags.csv')
after_firewall_all_2 = pd.read_csv('Data/Set 2/after_firewall_2_flags.csv')

In [119]:
before_firewall_all_3 = pd.read_csv('Data/Set 3/before_firewall_3_flags.csv')
after_firewall_all_3 = pd.read_csv('Data/Set 3/after_firewall_3_flags.csv')

In [120]:
before_firewall_all_4 = pd.read_csv('Data/Set 4/before_firewall_4_flags.csv')
after_firewall_all_4 = pd.read_csv('Data/Set 4/after_firewall_4_flags.csv')

In [121]:
before_firewall_all_5 = pd.read_csv('Data/Set 5/before_firewall_5_flags.csv')
after_firewall_all_5 = pd.read_csv('Data/Set 5/after_firewall_5_flags.csv')

In [122]:
before_firewall_all_6 = pd.read_csv('Data/Set 6/before_firewall_6_flags.csv')
after_firewall_all_6 = pd.read_csv('Data/Set 6/after_firewall_6_flags.csv')

In [123]:
before_firewall_all_7 = pd.read_csv('Data/Set 7/before_firewall_7_flags.csv')
after_firewall_all_7 = pd.read_csv('Data/Set 7/after_firewall_7_flags.csv')

In [124]:
before_firewall_all_8 = pd.read_csv('Data/Set 8/before_firewall_8_flags.csv')
after_firewall_all_8 = pd.read_csv('Data/Set 8/after_firewall_8_flags.csv')

## Automatization process...

In [111]:
def separate_packets(before_firewall, after_firewall):
    """Separates the rejected/dropped packets from the accepted ones"""
    
    # Drop duplicates from both dataframes
    before_firewall_no_duplicates = before_firewall.drop_duplicates()
    after_firewall_no_duplicates = after_firewall.drop_duplicates()
    
    # Left join to select only dropped/rejected packets
    df_left_join = pd.merge(before_firewall_no_duplicates, after_firewall_no_duplicates, 
                              on=['Source','Destination','Protocol','Source Port','Destination Port','Flags','Content Type'], 
                              how='left', 
                              indicator='Exist')
    
    # Separate the packets
    dropped_packets = df_left_join[df_left_join["Exist"]=="left_only"]
    accepted_packets = df_left_join[df_left_join["Exist"]=="both"]
    
    # Drop "Exist" column
    dropped_packets = dropped_packets.drop(columns=["Exist"])
    accepted_packets = accepted_packets.drop(columns=["Exist"])
    
    # Add label
    dropped_packets["Accepted"] = 0
    accepted_packets["Accepted"] = 1
    
    return dropped_packets, accepted_packets

In [149]:
after_firewall_all_8

,Source,Destination,Protocol,Source Port,Destination Port,Flags,Content Type
0,148.70.231.231,193.231.20.40,TCP,49453.0,80.0,0x010,NaN
1,148.70.231.231,193.231.20.40,HTTP/XML,49453.0,80.0,0x018,NaN
2,207.46.13.4,193.231.20.40,TCP,5020.0,80.0,0x0c2,NaN
3,207.46.13.4,193.231.20.40,TCP,5020.0,80.0,0x010,NaN
4,207.46.13.4,193.231.20.40,HTTP,5020.0,80.0,0x018,NaN
5,148.70.231.231,193.231.20.40,TCP,49453.0,80.0,0x010,NaN
6,148.70.231.231,193.231.20.40,TCP,49453.0,80.0,0x011,NaN
7,148.70.231.231,193.231.20.40,TCP,49471.0,80.0,0x002,NaN
8,207.46.13.4,193.231.20.40,TCP,5020.0,80.0,0x010,NaN
9,207.46.13.4,193.231.20.40,TCP,5020.0,80.0,0x011,NaN


In [126]:
df_packets = pd.DataFrame()

before_firewall_array = [before_firewall_all_1, before_firewall_all_2, before_firewall_all_3, 
                         before_firewall_all_4, before_firewall_all_5,before_firewall_all_6,
                        before_firewall_all_7,before_firewall_all_8]

after_firewall_array = [after_firewall_all_1, after_firewall_all_2,after_firewall_all_3,
                        after_firewall_all_4,after_firewall_all_5,after_firewall_all_6,
                       after_firewall_all_7,after_firewall_all_8]

In [127]:
def prepare_dataset(before_firewall_list, after_firewall_list):
    """Prepares the full dataset containing all the dropped and accepted packets with their labels"""
    df_rejected = pd.DataFrame()
    df_accepted = pd.DataFrame()

    for i in range(0, len(before_firewall_list)):
        dropped_pkts, accepted_pkts = separate_packets(before_firewall_list[i], after_firewall_list[i])
        df_rejected = pd.concat([df_rejected, dropped_pkts],axis=0).drop_duplicates()
        df_accepted = pd.concat([df_accepted, accepted_pkts], axis=0).drop_duplicates()
        
    return df_rejected, df_accepted

In [128]:
df_rejected, df_accepted = prepare_dataset(before_firewall_array,after_firewall_array)

In [129]:
df_rejected

,Source,Destination,Protocol,Source Port,Destination Port,Flags,Content Type,Accepted
21,104.238.118.103,193.231.20.40,TCP,48030.0,443.0,0x011,NaN,0
89,95.163.255.67,193.231.20.40,TCP,44497.0,443.0,0x011,NaN,0
117,192.0.78.33,193.231.20.40,TLSv1.2,443.0,36216.0,0x010,Handshake,0
126,54.146.176.100,193.231.20.40,TLSv1.2,443.0,60686.0,0x010,Handshake,0
132,95.163.255.65,193.231.20.40,TCP,59275.0,443.0,0x011,NaN,0
349,178.138.99.219,193.231.20.40,TLSv1.2,38682.0,443.0,0x019,Alert,0
413,69.46.36.14,193.231.20.40,TLSv1.2,443.0,39188.0,0x010,Handshake,0
602,192.0.78.32,193.231.20.40,TLSv1.2,443.0,47474.0,0x010,Handshake,0
655,69.46.36.14,193.231.20.40,TLSv1.2,443.0,39196.0,0x010,Handshake,0
809,192.0.78.33,193.231.20.40,TLSv1.2,443.0,36254.0,0x010,Handshake,0


In [130]:
df_accepted

,Source,Destination,Protocol,Source Port,Destination Port,Flags,Content Type,Accepted
0,164.132.230.244,193.231.20.40,TLSv1.2,38394.0,443.0,0x018,Application Data,1
1,82.77.127.28,193.231.20.40,TLSv1.2,55123.0,443.0,0x018,Alert,1
2,82.77.127.28,193.231.20.40,TCP,55123.0,443.0,0x011,NaN,1
3,82.77.127.28,193.231.20.40,TCP,55126.0,443.0,0x002,NaN,1
4,66.249.66.88,193.231.20.40,TCP,44678.0,80.0,0x002,NaN,1
5,82.77.127.28,193.231.20.40,TCP,55126.0,443.0,0x010,NaN,1
6,82.77.127.28,193.231.20.40,TLSv1.2,55126.0,443.0,0x018,Handshake,1
7,66.249.66.86,193.231.20.40,TCP,56993.0,80.0,0x002,NaN,1
8,164.132.230.244,193.231.20.40,TCP,38394.0,443.0,0x010,NaN,1
9,164.132.230.244,193.231.20.40,TLSv1.2,38394.0,443.0,0x018,Alert,1


We are doing another left merge between the rejected packets and the accepted packets to see if any duplcates slipped into both dataframes unnoticed.

In [154]:
df_left_join_testing = pd.merge(df_rejected, df_accepted, 
                              on=['Source','Destination','Protocol','Source Port','Destination Port','Flags','Content Type','Accepted'], 
                              how='left', 
                              indicator='IsDuplicated')

In [155]:
df_left_join_testing[df_left_join_testing['IsDuplicated']=="both"]

,Source,Destination,Protocol,Source Port,Destination Port,Flags,Content Type,Accepted,IsDuplicated
